# 時間序列資料分析與處理

本實作教學將主要關注時間序列分析的`數據整理`和`可視化`方面。通過使用時間序列的能源數據，我們將了解基於時間的索引編制，重採樣和滾動窗口之類的技術如何幫助我們探索電力需求和可再生能源供應隨時間的變化。我們將涵蓋以下主題：

* 範例資料集: Open Power Systems Data
* 時間序列數據結構
* Time-based 索引
* 時間序列數據可視化
* 季節性(Seasonality)
* 頻率(Frequencies)
* 重採樣(Resampling)
* 滾動時間窗口(Rolling windows)
* 趨勢(Trends)

## 範例資料集: Open Power Systems Data

我們使用德國的開放電源系統數據（OPSD）的每日時間序列。數據集包括2006-2017年德國全國電力消耗，風能生產和太陽能發電的總量。
電力生產和消耗報告為千兆瓦時（GWh）的每日總量。

資料: opsd_germany_daily.csv

數據文件的列為：
* Date — 日期 (yyyy-mm-dd format)
* Consumption — 電力消耗 (GWh)
* Wind — 風力發電 (GWh)
* Solar — 太陽能發電 (GWh)
* Wind+Solar — 風能和太陽能發電量之總和 (GWh)

我們將使用Pandas時間序列工具來回答以下問題，來探索德國的電力消耗和生產隨時間的變化：
* 通常什麼時候耗電量最高和最低？
* 風力和太陽能發電量會隨著季節的變化而變化嗎？
* 電力消耗，太陽能和風能的長期趨勢是什麼？
* 風能和太陽能發電與電能消耗如何比較，該比率隨時間變化如何？

## 時間序列數據結構

在深入研究OPSD數據之前，讓我們簡要介紹一下用於處理日期和時間的主要Pandas數據結構。在Pandas中，單個時間點表示為時間戳`Timestamp`。我們可以使用`to_datetime（）`函數根據各種日期/時間格式的字符串創建時間戳`Timestamp`。

In [ ]:
import pandas as pd

dt = pd.to_datetime('2018-01-15 3:45pm')

print(type(dt))
print(dt)

In [ ]:
dt = pd.to_datetime('7/8/1952')

print(type(dt))
print(dt)

如果我們提供一個字串列表或字串陣列作為`to_datetime（）`的輸入，它將在`DatetimeIndex`物件中返回日期/時間值的序列，`DatetimeIndex`物件是支持大多數Pandas時間序列功能的核心數據結構。

In [ ]:
dti = pd.to_datetime(['2018-01-05', '7/8/1952', 'Oct 10, 1995'])

print(type(dti))
print(dti)

## 創建時間序列DataFrame

為了使用Pandas來處理時間序列數據，我們需要使用`DatetimeIndex`作為`DataFrame`（或`Series`）的索引。讓我們看看如何使用我們的OPSD數據集執行此操作。首先，我們使用`read_csv（）`函數將數據讀取到DataFrame中，然後顯示其資料的shape。

In [ ]:
opsd_daily = pd.read_csv('data/opsd_germany_daily.csv')

print(opsd_daily.shape)
print(opsd_daily.info())
print(opsd_daily.head())


DataFrame有4383行，涵蓋從2006年1月1日到2017年12月31日的時間段。要查看數據的樣子，讓我們使用`head（）`和`tail（）`方法顯示前三行和最後三行。

In [ ]:
opsd_daily.head(3)

In [ ]:
opsd_daily.tail(3)

接下來，讓我們檢查每一列的數據類型。

In [ ]:
opsd_daily.dtypes

In [ ]:
cell = opsd_daily.iloc[0]['Date']
print(type(cell))
print(cell)

In [ ]:
opsd_daily['Date'] = pd.to_datetime(opsd_daily['Date'])

print(opsd_daily.info())

現在，`Date`欄己經是正確的數據類型，我們現在要將其設置為DataFrame的索引。

In [ ]:
opsd_daily = opsd_daily.set_index('Date')

print(opsd_daily.info())
print(opsd_daily.head())

In [ ]:
dti = opsd_daily.index

print(type(dti))
print(dti)

另外，我們可以使用`read_csv（）`函數的`index_col`和`parse_dates`參數將上述步驟合併一起執行。這通常是一個有用的快捷方式。

In [ ]:
opsd_daily = pd.read_csv('data/opsd_germany_daily.csv', index_col=0, parse_dates=True)

print(opsd_daily.info())
print(opsd_daily.head())

現在，我們的DataFrame索引是`DatetimeIndex`，我們可以使用所有Pandas基於時間的強大索引來處理和分析我們的數據，這將在以下部分中看到。`DatetimeIndex`的另一個有用方面是，各個日期/時間組件都可以用作屬性，例如年，月，日等。讓我們在opsd_daily中添加更多列，其中包含年，月和周日的名稱。

In [ ]:
# Add columns with year, month, and weekday name

opsd_daily['Year'] = opsd_daily.index.year
opsd_daily['Month'] = opsd_daily.index.month
opsd_daily['Weekday Name'] = opsd_daily.index.day_name()

# Display a random sampling of 5 rows
print(opsd_daily.sample(5, random_state=0))

Pandas時間序列的最強大和便捷的功能是在`time-based indexing`的索引下, 可以使用日期和時間直觀地組織和訪問我們的數據。使用基於時間的索引，我們可以使用日期/時間格式的字串透過`loc`的手法來在DataFrame中選擇數據。

例如，我們可以使用諸如`2017-08-10`之類的字串選擇特定日期的數據。

In [ ]:
data = opsd_daily.loc['2017-08-10']

print(type(data))

print(data)

我們還可以選擇特定的時間區段，例如'2014-01-20'到'2014-01-22'。與使用`loc`的基於標籤的常規索引一樣，切片包含兩個端點。

In [ ]:
data = opsd_daily.loc['2014-01-20':'2014-01-22']

print(type(data))

print(data)

Pandas時間序列的另一個非常方便的功能是部分字串索引**partial-string indexing**，在這裡我們可以選擇部分匹配給定字串的日期/時間。例如，我們可以使用`opsd_daily.loc['2006']`選擇2006年全年的數據，或使用`opsd_daily.loc['2012-02']`選擇2012年2月整個月。

In [ ]:
data = opsd_daily.loc['2012-02']

print(type(data))

print(data)

## 時間序列數據可視化

使用pandas和matplotlib，我們可以輕鬆地可視化我們的時間序列數據。我們將介紹一些範例和一些有用的自定義時序圖。首先，讓我們導入matplotlib。

In [ ]:
import matplotlib.pyplot as plt

我們將對圖表使用`seaborn`樣式，然後將預設圖表尺寸調整為適合時間序列繪圖的形狀。

In [ ]:
import seaborn as sns

# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(11, 4)})

讓我們使用DataFrame的`plot（）`方法創建德國每日用電量的全時序列的線圖。

In [ ]:
opsd_daily['Consumption'].plot(linewidth=0.5);

我們可以看到`plot（）`方法為x軸選擇了很好的刻度位置（每兩年）和標籤（年份），這很有幫助。但是，由於數據點太多，因此線圖比較擁擠且難以讀取。讓我們將數據繪製成點，然後查看`Solar`和`Wind`時間序列。

In [ ]:
cols_plot = ['Consumption', 'Solar', 'Wind']
axes = opsd_daily[cols_plot].plot(marker='.', alpha=0.5, linestyle='None', figsize=(11, 9), subplots=True)

for ax in axes:
    ax.set_ylabel('Daily Totals (GWh)')

透過圖型的展示, 我們已經可以看到一些有趣的patterns了：
* 電力消耗在冬季最高，大概是由於用電加熱和照明使用增加，在夏季最低。
* 夏季的太陽能產量最高，而陽光最為豐富，而冬季則最低。
* 風力發電在冬季最高，大概是由於強風和更頻繁的風暴，而在夏季最低。
* 多年來，風電生產似乎呈現出強勁的增長趨勢。

所有這三個時間序列都清楚地顯示出`週期性`（在時間序列分析中通常稱為`季節性 seasonality`），其中，模式會以規則的時間間隔一次又一次地重複。消費，太陽和風的時間序列在每年的時間尺度上在高值和低值之間振盪，這與一年中天氣的季節性變化相對應。

季節性也可能在其他時間範圍內發生。上圖顯示，德國的用電量每周可能有一些季節性變化，與工作日和周末相對應。讓我們繪製一年中的時間序列，以進行進一步調查。

In [ ]:
ax = opsd_daily.loc['2017', 'Consumption'].plot()
ax.set_ylabel('Daily Consumption (GWh)');

現在我們可以清楚地看到每週的波動。在此粒度級別上變得明顯的另一個有趣功能是1月初和12月下旬假期期間的用電量急劇下降。

讓我們進一步放大，看看一月和二月。

In [ ]:
ax = opsd_daily.loc['2017-01':'2017-02', 'Consumption'].plot(marker='o', linestyle='-')
ax.set_ylabel('Daily Consumption (GWh)');

正如我們所懷疑的，平日的消費量最高，而周末的消費量最低。

## 季節性(Seasonality)

接下來，讓我們用箱形圖進一步探索數據的季節性，使用seaborn的`boxplot（）`函數按不同的時間段對數據進行分組並顯示每個組的分佈。我們將首先按`月`對數據進行分組，以可視化年度季節性。

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(11, 10), sharex=True)

for name, ax in zip(['Consumption', 'Solar', 'Wind'], axes):
    sns.boxplot(data=opsd_daily, x='Month', y=name, ax=ax)
    ax.set_ylabel('GWh')
    ax.set_title(name)
    # Remove the automatic x-axis label from all but the bottom subplot
    if ax != axes[-1]:
        ax.set_xlabel('')

這些箱形圖確認了我們在較早的圖中看到的年度季節性，並提供了其他一些見解：
* 儘管冬季的用電量通常較高，而夏季則較低，但是與11月和2月相比，12月和1月的四分位數中位數和下四分位數較低。我們在2017年的時間序列中看到了這一點，箱形圖證實了這是多年來一致的模式。
* 雖然太陽能和風能生產均顯示每年的季節性，但風能分佈卻有更多異常值，反映了暴風雨和其他瞬態天氣條件偶發的極端風速的影響。

接下來，讓我們按`星期幾`對耗電時間序列進行分組，以探討每週的季節性。

In [ ]:
sns.boxplot(data=opsd_daily, x='Weekday Name', y='Consumption');

## 頻率(Frequencies)

當時間序列的數據點在時間上均勻間隔（例如每小時，每天，每月等）時，該時間序列可以與Pandas的時間單位的頻率相關聯。例如，讓我們使用`date_range（）`函數以每天的頻率創建一個從`1998-03-10`到`1998-03-15`的均勻間隔的日期序列。

下表總結了可用的主要時間頻率代碼：


|Code	|Description	|Code	|Description|
|:------|:--------------|:------|:----------|
|`D`	|Calendar day	|`B`	|Business day|
|`W`	|Weekly|||		
|`M`	|Month end	|`BM`	|Business month end|
|`Q`	|Quarter end	|`BQ`	|Business quarter end|
|`A`	|Year end	|`BA`	|Business year end|
|`H`	|Hours	|`BH`	|Business hours|
|`T`	|Minutes|||				
|`S`	|Seconds|||				
|`L`	|Milliseonds|||				
|`U`	|Microseconds|||				
|`N`	|nanoseconds|||			

In [ ]:
dti = pd.date_range('1998-03-10', '1998-03-15', freq='D')

print(type(dti))
print(dti)

再舉一個例子，讓我們以每小時的頻率創建一個日期範圍，指定開始日期和期間數，而不是開始日期和結束日期。

In [ ]:
dti = pd.date_range('2004-09-20', periods=8, freq='H')

print(type(dti))
print(dti)

現在，讓我們再來看一下opsd_daily時間序列的DatetimeIndex。

In [ ]:
print(opsd_daily.index)

我們可以看到它沒有頻率`（freq=None）`的標示。因為該索引是從CSV文件中的日期序列創建而成的，因此它並沒有為時間序列明確指定任何`頻率`。

如果我們知道我們的數據應處於特定頻率，則可以使用DataFrame的`asfreq（）`方法來進行`頻率`的指定。如果數據中缺少任何日期/時間的slots，則Pandas將為這些日期/時間slots添加新的row，這些row可以為空（`NaN`），也可以根據指定的數據的手法來進行填充（例如正向填充或插值）。

為了了解其工作原理，讓我們創建一個新的DataFrame，其中僅包含2013年2月3日，6日和8日的數據。

In [ ]:
# To select an arbitrary sequence of date/time values from a pandas time series,
# we need to use a DatetimeIndex, rather than simply a list of date/time strings
times_sample = pd.to_datetime(['2013-02-03', '2013-02-06', '2013-02-08'])

# Select the specified dates and just the consumption column
consum_sample = opsd_daily.loc[times_sample, ['Consumption']].copy()

print(consum_sample.info())
print(consum_sample)

現在，我們使用`asfreq（）`方法將DataFrame轉換為每日頻率，其中一列用於未填充的數據，一列用於前向填充的數據。

In [ ]:
# Convert the data to daily frequency, without filling any missings
consum_freq = consum_sample.asfreq('D')

print(consum_freq)

In [ ]:
# Create a column with missings forward filled
consum_freq['Consumption - Forward Fill'] = consum_sample.asfreq('D', method='ffill')

print(consum_freq)

在`Consumption`列中，我們有原始數據。而'NaN'的資料透過`ffill`的手法來補值，這意味著最後一個值會在缺失的行中重複，直到出現下一個非缺失值。

如果你要進行任何需要均勻間隔的數據且沒有任何遺漏的時間序列分析，則需要使用`asfreq（）`將時間序列轉換為指定的頻率，並使用適當的方法填充所有遺漏值。

## 重採樣(Resampling)

將我們的時間序列數據`重新採樣`到較低或較高的頻率是很常被使用的功能。重採樣到較低的頻率（downsampling）通常涉及聚合操作-例如，根據`每日`銷售總額數據計算`每月`的銷售總額。

我們在本教學中使用的每日OPSD數據是從原始的每小時時間序列中縮減採樣的。重採樣到較高頻率（upsamping）的情況比較少見，並且通常涉及插值或其他數據填充方法，例如，將每小時天氣數據插值到10分鐘間隔以輸入到科學模型中。

我們將在這裡著重於downsampling，探索它如何幫助我們在各種時間尺度上分析我們的OPSD數據。我們使用DataFrame的`resample（）`方法，該方法將DatetimeIndex拆分為多個時間段，並按時間段對數據進行分組。 `resample（）`方法返回一個`Resampler`物件，類似於pandas `GroupBy`物件。然後，我們可以對每個時間段的數據組應用聚合方法，例如mean(), median(), sum()...等等。

例如，讓我們將數據重新採樣為每周平均時間序列（`Day` -> `Week`):

In [ ]:
# Specify the data columns we want to include (i.e. exclude Year, Month, Weekday Name)
data_columns = ['Consumption', 'Wind', 'Solar', 'Wind+Solar']

# Resample to weekly frequency, aggregating with mean
opsd_weekly_mean = opsd_daily[data_columns].resample('W').mean()

print(opsd_weekly_mean.info())

print(opsd_weekly_mean.head(3))

上面標記為2006-01-01的第一行包含時間範圍2006-01-01至2006-01-07中包含的所有數據的平均值。第二行標記為2006-01-08，其中包含2006-01-08至2006-01-14時間段的平均值數據，依此類推。

我們的**每週**時間序列的數據點是**每日**時間序列的1/7。我們可以通過比較兩個DataFrame的行數來確認這一點。

In [ ]:
print(opsd_daily.shape[0])
print(opsd_weekly_mean.shape[0])

讓我們一起繪製一個六個月內的每日和每週的`Solar`時間序列，以進行比較。

In [ ]:
# Start and end of the date range to extract
start, end = '2017-01', '2017-06'

# Plot daily and weekly resampled time series together
fig, ax = plt.subplots()

ax.plot(opsd_daily.loc[start:end, 'Solar'],
marker='.', linestyle='-', linewidth=0.5, label='Daily')

ax.plot(opsd_weekly_mean.loc[start:end, 'Solar'],
marker='o', markersize=8, linestyle='-', label='Weekly Mean Resample')

ax.set_ylabel('Solar Production (GWh)')
ax.legend();

我們可以看到，每周平均時間序列比每日時間序列平滑，這是因為在重採樣中平均了較高的頻率變異性。

現在，我們將數據重新採樣到`每月`一次，匯總`總和`而不是`均值`。與使用`mean（）`進行聚合（將所有缺少數據的任何時間段的輸出均設置為NaN）不同，`sum（）`的預設行為將輸出0作為缺失數據的總和。我們使用`min_count`參數來更改此行為。

In [ ]:
# Compute the monthly sums, setting the value to NaN for any month which has
# fewer than 28 days of data
opsd_monthly = opsd_daily[data_columns].resample('M').sum(min_count=28)
opsd_monthly.head(3)

現在，我們將用電量繪製成線形圖，將風能和太陽能發電量繪製成堆積面積圖，從而探索每月的時間序列。

In [ ]:
fig, ax = plt.subplots()

ax.plot(opsd_monthly['Consumption'], color='black', label='Consumption')

opsd_monthly[['Wind', 'Solar']].plot.area(ax=ax, linewidth=0)

ax.legend()

ax.set_ylabel('Monthly Total (GWh)');

在這個`月`的時間尺度上，我們可以清楚地看到每個時間序列中的年度`季節性`，並且很明顯，用電量隨著時間的推移一直相當穩定，而風電產量一直在穩定增長，其中風能+太陽能發電量不斷增加消耗的電力份額。

讓我們通過重新採樣到`年度頻率`並且計算每年的風能+太陽能與消耗量之比率。

In [ ]:
# Compute the annual sums, setting the value to NaN for any year which has
# fewer than 360 days of data
opsd_annual = opsd_daily[data_columns].resample('A').sum(min_count=360)
# The default index of the resampled DataFrame is the last day of each year,
# ('2006-12-31', '2007-12-31', etc.) so to make life easier, set the index
# to the year component
opsd_annual = opsd_annual.set_index(opsd_annual.index.year)
opsd_annual.index.name = 'Year'

opsd_annual

In [ ]:
# Compute the ratio of Wind+Solar to Consumption
opsd_annual['Wind+Solar/Consumption'] = opsd_annual['Wind+Solar'] / opsd_annual['Consumption']
opsd_annual.tail(3)

最後，讓我們用條形圖繪製風電與太陽能在年度用電量中所佔的比例。

In [ ]:
# Plot from 2012 onwards, because there is no solar production data in earlier years
ax = opsd_annual.loc[2012:, 'Wind+Solar/Consumption'].plot.bar(color='C0')
ax.set_ylabel('Fraction')
ax.set_ylim(0, 0.3)
ax.set_title('Wind + Solar Share of Annual Electricity Consumption')
plt.xticks(rotation=0);

我們可以看到，風能+太陽能生產佔年度用電量的比例已從2012年的約15％增長到2017年的約27％。

## 滾動時間窗口(Rolling windows)

滾動窗口操作是時間序列數據的另一個重要轉換。與down-sampling類似，滾動窗口將數據拆分為`時間窗口`，並且每個窗口中的數據都使用mean(), median(), sum()等函數進行匯總。不會重疊，並且輸出的頻率低於輸入的頻率，滾動窗口會以與數據相同的頻率重疊並“**滾動**”，因此變換後的時間序列與原始時間序列的頻率相同。

但是，與down-sampling不同，在down-sampling時間段不重疊並且輸出的頻率低於輸入的頻率，滾動窗口時間會重疊，並且以與數據相同的頻率“滾動”，因此變換後的時間序列處於相同的頻率作為原始時間序列。

讓我們使用`rolling（）`方法來計算每日數據的`7天滾動平均值`。我們使用center=True參數標記每個窗口的中點，因此滾動窗口為：

* `2006-01-01` to `2006-01-07` — labelled as 2006-01-04
* `2006-01-02` to `2006-01-08` — labelled as 2006-01-05
* `2006-01-03` to `2006-01-09` — labelled as 2006-01-06
* and so on…

In [ ]:
# Compute the centered 7-day rolling mean
data_columns = ['Consumption', 'Wind', 'Solar', 'Wind+Solar']

opsd_7d = opsd_daily[data_columns].rolling(7, center=True).mean()
opsd_7d.head(10)

我們可以看到第一個非丟失滾動平均值在2006-01-04上，因為這是第一個滾動窗口的中點。

為了形象化滾動平均值和重採樣之間的差異，讓我們更新我們較早的2017年1月至6月太陽能發電量圖，包括7天滾動平均值，每周平均重採樣時間序列和原始每日數據。

In [ ]:
# Start and end of the date range to extract
start, end = '2017-01', '2017-06'

# Plot daily, weekly resampled, and 7-day rolling mean time series together
fig, ax = plt.subplots()

ax.plot(opsd_daily.loc[start:end, 'Solar'], marker='.', linestyle='-', linewidth=0.5, label='Daily')

ax.plot(opsd_weekly_mean.loc[start:end, 'Solar'], marker='o', markersize=8, linestyle='-', label='Weekly Mean Resample')

ax.plot(opsd_7d.loc[start:end, 'Solar'], marker='.', linestyle='-', label='7-d Rolling Mean')

ax.set_ylabel('Solar Production (GWh)')
ax.legend();

## 趨勢(Trends)

除了較高的頻率可變性（例如季節性和噪聲）之外，時間序列數據通常還表現出一些緩慢的`漸進可變性`。可視化這些趨勢的一種簡單方法是在不同的時間範圍內使用滾動方式。

滾動平均值趨向於通過平均遠高於窗口大小的頻率上的變化並平均等於窗口大小的時間尺度上的任何季節性來平滑時間序列。這允許探索數據中的低頻變化。由於我們的用電時間序列具有每周和每年的季節性，因此讓我們來看一下這兩個時間尺度上的滾動平均值。

我們已經計算了7天的滾動平均值，因此現在我們計算OPSD數據的365天的滾動平均值。


In [ ]:
# The min_periods=360 argument accounts for a few isolated missing days in the
# wind and solar production time series
opsd_365d = opsd_daily[data_columns].rolling(window=365, center=True, min_periods=360).mean()

讓我們繪製7天和365天的滾動平均用電量以及每日時間序列。

In [ ]:
# Plot daily, 7-day rolling mean, and 365-day rolling mean time series
fig, ax = plt.subplots()

ax.plot(opsd_daily['Consumption'], marker='.', markersize=2, color='0.6',linestyle='None', label='Daily')

ax.plot(opsd_7d['Consumption'], linewidth=2, label='7-d Rolling Mean')

ax.plot(opsd_365d['Consumption'], color='0.2', linewidth=3,label='Trend (365-d Rolling Mean)')

ax.legend()
ax.set_xlabel('Year')
ax.set_ylabel('Consumption (GWh)')
ax.set_title('Trends in Electricity Consumption');

我們可以看到，為期7天的滾動均值使所有的每周季節性都變得平滑，同時保留了年度季節性。 7天的滾動平均值顯示，雖然冬季的用電量通常較高，而夏季則較低，但每個冬季在12月底和1月初的節假日期間，其用電量會急劇下降數週。

縱觀365天的滾動平均時間序列，我們可以看到電力消耗的長期趨勢相當平穩，在2009年和2012-2013年之間有幾個異常的低電量消耗時期。

現在，讓我們看一下風能和太陽能生產的趨勢。

In [ ]:
# Plot 365-day rolling mean time series of wind and solar power
fig, ax = plt.subplots()

for nm in ['Wind', 'Solar', 'Wind+Solar']:
    ax.plot(opsd_365d[nm], label=nm)
    ax.set_ylim(0, 400)
    ax.legend()
    ax.set_ylabel('Production (GWh)')
    ax.set_title('Trends in Electricity Production (365-d Rolling Means)');

隨著德國繼續擴大其在這些領域的能力，我們可以看到太陽能發電的增長趨勢較小，而風力發電的增長趨勢較大。